In [1]:
import os
import boto3
import csv

In [5]:
AWS_KEY_ID = os.getenv("AWS_KEY_ID") 
AWS_KEY = os.getenv("AWS_KEY") 
DATA_CONT_NAME = 'cloudcomputingdatacont'

In [7]:
s3 = boto3.resource('s3',
                    aws_access_key_id=AWS_KEY_ID,
                    aws_secret_access_key=AWS_KEY)

try:
    s3.create_bucket(Bucket=DATA_CONT_NAME)
except Exception as e:
    print(e)

bucket = s3.Bucket(DATA_CONT_NAME)
bucket.Acl().put(ACL='public-read')

body = open('./exp1.csv', 'rb')
o = s3.Object(DATA_CONT_NAME, 'test').put(Body=body)
s3.Object(DATA_CONT_NAME, 'test').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'XGG6EAVM8NQJDTCN',
  'HostId': '8pkEzgeJ7Y17kNY04XooDwoneThucvBSlwKdhKzxiK6PAz3D+7S5rE/Ko6F9Ogrsu5jQ2ATro00=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8pkEzgeJ7Y17kNY04XooDwoneThucvBSlwKdhKzxiK6PAz3D+7S5rE/Ko6F9Ogrsu5jQ2ATro00=',
   'x-amz-request-id': 'XGG6EAVM8NQJDTCN',
   'date': 'Fri, 22 Oct 2021 02:53:23 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [8]:
dyndb = boto3.resource('dynamodb',
                       region_name='us-east-1',
                       aws_access_key_id=AWS_KEY_ID, aws_secret_access_key=AWS_KEY
                       )

In [9]:
try:
    table = dyndb.create_table(
        TableName='DataTable', 
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            }, 
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ], 
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            }, 
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ], 
        ProvisionedThroughput=
        {
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except Exception as e:
    print (e)
    #if there is an exception, the table may already exist.
    table = dyndb.Table("DataTable")

In [10]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [11]:
print(table.item_count)

0


In [15]:
with open('experiments.csv', 'r') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)
    for item in csvf:
        print(item)
        body = open('./'+ item[4], 'rb')
        s3.Object(DATA_CONT_NAME, item[4]).put(Body=body)
        md = s3.Object(DATA_CONT_NAME, item[4]).Acl().put(ACL='public-read')
        url = "https://s3-us-west-2.amazonaws.com/njiang765/"+item[4]
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],
                         'Conductivity' : item[2], 'Concentration': item[3], 'url':url}
        try:
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [16]:
response = table.get_item(
    Key={
        'PartitionKey': '2',
        'RowKey': '-2'
    }
)
item = response['Item']

In [17]:
print(item)

{'Concentration': '3.4', 'PartitionKey': '2', 'RowKey': '-2', 'url': 'https://s3-us-west-2.amazonaws.com/njiang765/exp2.csv', 'Conductivity': '52.1'}


In [18]:
response = table.get_item(
    Key={
        'PartitionKey': '1',
        'RowKey': '-1'
    }
)
item = response['Item']

In [19]:
print(item)

{'Concentration': '3.4', 'PartitionKey': '1', 'RowKey': '-1', 'url': 'https://s3-us-west-2.amazonaws.com/njiang765/exp1.csv', 'Conductivity': '52'}
